IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M3\FAULT_M3_5


In [3]:
guasto=1

VOLO m3 FAULT 5%

In [4]:
rcou_m3_fault5 = pd.read_csv("RCOU.csv")
rcou_m3_fault5 = rcou_m3_fault5.astype("float64")
rcou_m3_fault5 = rcou_m3_fault5.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_fault5 = rcou_m3_fault5[((rcou_m3_fault5['C9'] >= potenza) & (rcou_m3_fault5['C10'] >= potenza) & (rcou_m3_fault5['C11'] >= potenza) & (rcou_m3_fault5['C12'] >= potenza) & (rcou_m3_fault5['C13'] >= potenza) & (rcou_m3_fault5['C14']>= potenza))]
rcou_m3_fault5=rcou_m3_fault5.reset_index(drop=True)
display(rcou_m3_fault5)

,TimeUS,C9,C10,C11,C12,C13,C14
0,204202091.0,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0
1,204212099.0,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0
2,204222805.0,1422.0,1454.0,1486.0,1386.0,1470.0,1405.0
3,204233384.0,1471.0,1448.0,1448.0,1471.0,1457.0,1462.0
4,204243252.0,1509.0,1450.0,1435.0,1522.0,1463.0,1497.0
...,...,...,...,...,...,...,...
8582,303987906.0,1536.0,1428.0,1542.0,1421.0,1443.0,1524.0
8583,303999574.0,1533.0,1428.0,1529.0,1433.0,1427.0,1534.0
8584,304010535.0,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0
8585,304021075.0,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0


In [5]:
min=rcou_m3_fault5['TimeUS'][0]
max=rcou_m3_fault5['TimeUS'][len(rcou_m3_fault5)-1]
print(max)
print(min)

304031184.0
204202091.0


In [6]:
xkf1_m3_fault5 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault5 = xkf1_m3_fault5.astype("float64")
xkf1_m3_fault5 = xkf1_m3_fault5.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault5 = xkf1_m3_fault5[((xkf1_m3_fault5['TimeUS'] >= min) & (xkf1_m3_fault5['TimeUS'] <= max))]
xkf1_m3_fault5 = xkf1_m3_fault5.reset_index(drop=True)
print(xkf1_m3_fault5)

           TimeUS  Roll  Pitch   Yaw
0     204210668.0  0.88  -0.77  2.52
1     204220693.0  0.85  -0.83  2.51
2     204230930.0  0.91  -0.79  2.53
3     204241781.0  0.94  -0.72  2.56
4     204251811.0  1.02  -0.76  2.56
...           ...   ...    ...   ...
9085  303985816.0  1.10  -0.30  2.01
9086  303997185.0  1.06  -0.25  2.00
9087  304007782.0  0.97  -0.22  1.97
9088  304018840.0  0.93  -0.14  1.94
9089  304029034.0  0.96  -0.06  1.90

[9090 rows x 4 columns]


In [7]:
att_m3_fault5 = pd.read_csv("ATT.csv")
att_m3_fault5 = att_m3_fault5.astype("float64")
att_m3_fault5 = att_m3_fault5.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_fault5 = att_m3_fault5[((att_m3_fault5['TimeUS'] >= min) & (att_m3_fault5['TimeUS'] <= max	))]
att_m3_fault5=att_m3_fault5.reset_index(drop=True)
print(att_m3_fault5)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw   Yaw
0     204211309.0     0.00  0.88      0.00  -0.77    2.49  2.52
1     204222059.0     0.00  0.85      0.00  -0.83    2.49  2.51
2     204232678.0     0.00  0.91      0.00  -0.79    2.49  2.53
3     204242578.0     0.00  0.94      0.00  -0.72    2.49  2.56
4     204253192.0     0.00  1.02      0.00  -0.76    2.49  2.56
...           ...      ...   ...       ...    ...     ...   ...
9086  303987111.0     2.69  1.10      1.47  -0.30    1.71  2.01
9087  303998743.0     2.66  1.06      1.49  -0.25    1.71  2.00
9088  304009828.0     2.65  0.97      1.52  -0.22    1.71  1.97
9089  304020291.0     2.64  0.93      1.55  -0.14    1.71  1.94
9090  304030671.0     2.63  0.96      1.58  -0.06    1.71  1.90

[9091 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_fault5)>len(xkf1_m3_fault5):
    to_add=att_m3_fault5[len(xkf1_m3_fault5):]
    att_m3_fault5=att_m3_fault5[:len(xkf1_m3_fault5)]

if len(xkf1_m3_fault5)>len(att_m3_fault5):
    to_add=xkf1_m3_fault5[len(att_m3_fault5):]

for idx,i in enumerate(att_m3_fault5['Roll']):
    if(xkf1_m3_fault5['Roll'][idx] != i):
        att_m3_fault5['Roll'][idx] = ((att_m3_fault5['Roll'][idx] + xkf1_m3_fault5['Roll'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Pitch']):
    if(xkf1_m3_fault5['Pitch'][idx] != i):
        att_m3_fault5['Pitch'][idx] = ((att_m3_fault5['Pitch'][idx] + xkf1_m3_fault5['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_fault5['Yaw']):
    if(xkf1_m3_fault5['Yaw'][idx] != i):
        att_m3_fault5['Yaw'][idx] = ((att_m3_fault5['Yaw'][idx] + xkf1_m3_fault5['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_fault5=pd.concat([att_m3_fault5,to_add])

print(att_m3_fault5)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw
0     204211309.0     0.00  0.880      0.00 -0.770    2.49  2.520
1     204222059.0     0.00  0.850      0.00 -0.830    2.49  2.510
2     204232678.0     0.00  0.910      0.00 -0.790    2.49  2.530
3     204242578.0     0.00  0.940      0.00 -0.720    2.49  2.560
4     204253192.0     0.00  1.020      0.00 -0.760    2.49  2.560
...           ...      ...    ...       ...    ...     ...    ...
9086  303987111.0     2.69  1.080      1.47 -0.275    1.71  2.005
9087  303998743.0     2.66  1.015      1.49 -0.235    1.71  1.985
9088  304009828.0     2.65  0.950      1.52 -0.180    1.71  1.955
9089  304020291.0     2.64  0.945      1.55 -0.100    1.71  1.920
9090  304030671.0     2.63  0.960      1.58 -0.060    1.71  1.900

[9091 rows x 7 columns]


In [9]:
'''#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_fault5 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault5 = esc_0_m3_fault5.astype("float64")
esc_0_m3_fault5 = esc_0_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_fault5 = esc_0_m3_fault5[((esc_0_m3_fault5['TimeUS'] >= min) & (esc_0_m3_fault5['TimeUS'] <= max))]
esc_0_m3_fault5=esc_0_m3_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_fault5 = pd.read_csv("ESC_1.csv")
esc_1_m3_fault5 = esc_1_m3_fault5.astype("float64")
esc_1_m3_fault5 = esc_1_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_fault5 = esc_1_m3_fault5[((esc_1_m3_fault5['TimeUS'] >= min) & (esc_1_m3_fault5['TimeUS'] <= max))]
esc_1_m3_fault5=esc_1_m3_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_fault5 = pd.read_csv("ESC_2.csv")
esc_2_m3_fault5 = esc_2_m3_fault5.astype("float64")
esc_2_m3_fault5 = esc_2_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_fault5 = esc_2_m3_fault5[((esc_2_m3_fault5['TimeUS'] >= min) & (esc_2_m3_fault5['TimeUS'] <= max))]
esc_2_m3_fault5=esc_2_m3_fault5.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_fault5 = pd.read_csv("ESC_3.csv")
esc_3_m3_fault5 = esc_3_m3_fault5.astype("float64")
esc_3_m3_fault5 = esc_3_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_fault5 = esc_3_m3_fault5[((esc_3_m3_fault5['TimeUS'] >= min) & (esc_3_m3_fault5['TimeUS'] <= max))]
esc_3_m3_fault5=esc_3_m3_fault5.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_fault5 = pd.read_csv("ESC_4.csv")
esc_4_m3_fault5 = esc_4_m3_fault5.astype("float64")
esc_4_m3_fault5 = esc_4_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_fault5 = esc_4_m3_fault5[((esc_4_m3_fault5['TimeUS'] >= min) & (esc_4_m3_fault5['TimeUS'] <= max))]
esc_4_m3_fault5=esc_4_m3_fault5.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_fault5 = pd.read_csv("ESC_5.csv")
esc_5_m3_fault5 = esc_5_m3_fault5.astype("float64")
esc_5_m3_fault5 = esc_5_m3_fault5.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_fault5 = esc_5_m3_fault5[((esc_5_m3_fault5['TimeUS'] >= min) & (esc_5_m3_fault5['TimeUS'] <= max))]
esc_5_m3_fault5=esc_5_m3_fault5.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})'''

'#elettricita\' che scorre nel motore considerato, al momento ne consideriamo solo 1\nesc_0_m3_fault5 = pd.read_csv("ESC_0.csv")\nesc_0_m3_fault5 = esc_0_m3_fault5.astype("float64")\nesc_0_m3_fault5 = esc_0_m3_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_0_m3_fault5 = esc_0_m3_fault5[((esc_0_m3_fault5[\'TimeUS\'] >= min) & (esc_0_m3_fault5[\'TimeUS\'] <= max))]\nesc_0_m3_fault5=esc_0_m3_fault5.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})\n\nesc_1_m3_fault5 = pd.read_csv("ESC_1.csv")\nesc_1_m3_fault5 = esc_1_m3_fault5.astype("float64")\nesc_1_m3_fault5 = esc_1_m3_fault5.drop([\'LineNo\',\'Instance\', \'RawRPM\', \'Volt\', \'Temp\', \'CTot\', \'MotTemp\', \'Err\'], axis=1)\nesc_1_m3_fault5 = esc_1_m3_fault5[((esc_1_m3_fault5[\'TimeUS\'] >= min) & (esc_1_m3_fault5[\'TimeUS\'] <= max))]\nesc_1_m3_fault5=esc_1_m3_fault5.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})\n\nesc_2_m3_fault5 = pd.r

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_fault5 = pd.read_csv("IMU_0.csv")
imu_0_m3_fault5 = imu_0_m3_fault5.astype("float64")
imu_0_m3_fault5 = imu_0_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_fault5 = imu_0_m3_fault5[((imu_0_m3_fault5['TimeUS'] >= min) & (imu_0_m3_fault5['TimeUS'] <= max))]

imu_1_m3_fault5 = pd.read_csv("IMU_1.csv")
imu_1_m3_fault5 = imu_1_m3_fault5.astype("float64")
imu_1_m3_fault5 = imu_1_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_fault5 = imu_1_m3_fault5[((imu_1_m3_fault5['TimeUS'] >= min) & (imu_1_m3_fault5['TimeUS'] <= max))]

imu_2_m3_fault5 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault5 = imu_2_m3_fault5.astype("float64")
imu_2_m3_fault5 = imu_2_m3_fault5.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_fault5 = imu_2_m3_fault5[((imu_2_m3_fault5['TimeUS'] >= min) & (imu_2_m3_fault5['TimeUS'] <= max))]

imu_m3_fault5 = pd.concat((imu_0_m3_fault5, imu_1_m3_fault5, imu_2_m3_fault5))
imu_m3_fault5=imu_m3_fault5.groupby(imu_m3_fault5.TimeUS, as_index=False).mean()

display(imu_m3_fault5)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,204204632.0,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536
1,204206099.0,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001
2,204209242.0,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777
3,204211153.0,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143
4,204213901.0,-0.035754,0.064878,0.045289,-0.319344,0.122338,-10.160297
...,...,...,...,...,...,...,...
34503,304020133.0,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323
34504,304022922.0,0.050244,0.063406,-0.067958,0.036619,0.395237,-10.919890
34505,304025102.0,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300
34506,304027773.0,-0.008855,0.096466,-0.051342,-0.315915,0.138011,-10.782627


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_fault5 = pd.merge_ordered(imu_m3_fault5,att_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
'''m3_fault5 = pd.merge_ordered(m3_fault5, esc_0_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_1_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_2_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_3_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_4_m3_fault5)
m3_fault5 = pd.merge_ordered(m3_fault5, esc_5_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")'''
m3_fault5 = pd.merge_ordered(m3_fault5, rcou_m3_fault5)
m3_fault5=m3_fault5.fillna(method="ffill").fillna(method="bfill")

In [12]:
m3_fault5["TimeUS"] = m3_fault5["TimeUS"] - m3_fault5.iloc[0]["TimeUS"]
m3_fault5["TimeUS"] = m3_fault5["TimeUS"].astype(int)
m3_fault5["Guasto"] = guasto

In [13]:
last_cell = m3_fault5.iloc[-1, m3_fault5.columns.get_loc('TimeUS')]
print(last_cell)

99829093


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
34937  99815009
34938  99817866
34939  99820723
34940  99823580
34941  99826437

[34942 rows x 1 columns]


In [16]:
m3_fault5_final = pd.merge_ordered(m3_fault5,df)
m3_fault5_final=m3_fault5_final.fillna(method="ffill").fillna(method="bfill")
print(m3_fault5_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
1          2541  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
2          2857  0.115041 -0.156477 -0.036312  0.172309 -0.134253  -9.789536   
3          4008  0.026523 -0.083041  0.008797 -0.019900 -0.217393  -9.794001   
4          5714  0.026523 -0.083041  0.008797 -0.019900 -0.217393  -9.794001   
...         ...       ...       ...       ...       ...       ...        ...   
87100  99825682 -0.008855  0.096466 -0.051342 -0.315915  0.138011 -10.782627   
87101  99826437 -0.008855  0.096466 -0.051342 -0.315915  0.138011 -10.782627   
87102  99828166 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   
87103  99828580 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   
87104  99829093 -0.052177  0.183527 -0.037859 -0.200246  0.199173 -10.854337   

       DesRoll   Roll  DesPitch  Pitch 

In [17]:
m3_fault5_final = m3_fault5_final[m3_fault5_final.TimeUS.isin(to_be)]

In [18]:
m3_fault5_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,0,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
2,2857,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
4,5714,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
6,8571,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
10,11428,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143,0.00,0.880,0.00,-0.77,2.49,2.520,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87091,99815009,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,-0.18,1.71,1.955,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
87092,99817866,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,-0.18,1.71,1.955,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
87096,99820723,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323,2.64,0.945,1.55,-0.10,1.71,1.920,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0
87099,99823580,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300,2.64,0.945,1.55,-0.10,1.71,1.920,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0


In [19]:
from datetime import timedelta
m3_fault5_final=m3_fault5_final.reset_index(drop=True)
m3_fault5_final['TimeUS'] = pd.to_datetime(m3_fault5_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_fault5_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
1,00:00:00.002857,0.115041,-0.156477,-0.036312,0.172309,-0.134253,-9.789536,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
2,00:00:00.005714,0.026523,-0.083041,0.008797,-0.019900,-0.217393,-9.794001,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
3,00:00:00.008571,-0.143925,0.062323,0.064051,-0.122822,-0.059922,-10.048777,0.00,0.880,0.00,-0.77,2.49,2.520,1406.0,1380.0,1413.0,1372.0,1424.0,1359.0,1.0
4,00:00:00.011428,-0.172116,0.091631,0.076426,-0.149506,0.052869,-10.198143,0.00,0.880,0.00,-0.77,2.49,2.520,1369.0,1459.0,1408.0,1424.0,1358.0,1468.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34937,00:01:39.815009,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,-0.18,1.71,1.955,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
34938,00:01:39.817866,-0.041717,0.132493,-0.056576,-0.175003,0.174827,-10.644793,2.65,0.950,1.52,-0.18,1.71,1.955,1532.0,1424.0,1521.0,1437.0,1419.0,1537.0,1.0
34939,00:01:39.820723,-0.043775,0.158520,-0.049635,-0.113556,0.259399,-10.788323,2.64,0.945,1.55,-0.10,1.71,1.920,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0
34940,00:01:39.823580,0.065195,0.030756,-0.069719,-0.167383,0.277658,-10.851300,2.64,0.945,1.55,-0.10,1.71,1.920,1523.0,1428.0,1498.0,1456.0,1385.0,1557.0,1.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_fault5_final)/350)):
        V1=m3_fault5_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
'''rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')'''
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')
#merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5,c_9,c_10,c_11,c_12,c_13,c_14]
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_10160\162550328.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_10160\162550328.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,0.010773,0.007645,-0.477744,0.087972,75.0,62.0,1.783221,1.423987,0.023235,0.005472,-0.378847,0.077435,92.0,72.0,1.949840,1.374847,0.146326,0.006021,-0.875847,0.165574,82.0,83.0,0.841562,0.676066,0.019303,0.036670,0.760214,0.192194,110.0,80.0,6.510026,6.375686,0.298050,0.038849,0.047521,0.357172,97.0,100.0,17.109015,3.023219,-10.867889,3.072157,-1.628420,11.007925,0.0,120.0,200.480931,46.564065,2.038376,0.378622,-1.062367,2.128972,92.0,89.0,16.937560,1.837580,0.090456,0.234335,-0.669611,0.491782,103.0,102.0,23.662019,4.632788,6.478946,8.791587,-1.546589,7.123432,122.0,88.0,23.226543,-1.973571,1588.826211,3019.983997,3.262358,1589.773603,117.0,118.0,789.066020,778.085453,1480.538462,6321.266374,-1.567264,1482.665638,130.0,131.0,1187.743067,406.143047,1564.381766,1989.916695,2.480789,1565.015833,123.0,117.0,882.946316,406.608109,1509.025641,6468.607912,-1.616902,1511.161330,122.0,121.0,699.854949,424.787283,1501.441595,6173.190151,-1.565622,1503.490095,119.0,117.0,1134.651910,997.735233,1570.843305,2727.486805,2.648922,1571.708753,114.0,115.0,502.371299,309.894381,1
1,1.0,0.010485,0.007652,-0.481629,0.087979,58.0,74.0,1.881163,1.705376,0.022757,0.005537,-0.377164,0.077710,88.0,70.0,1.867390,1.400903,0.146155,0.006014,-0.877579,0.165402,86.0,87.0,0.850265,0.649072,0.020405,0.036993,0.749206,0.193143,122.0,118.0,6.372392,5.871604,0.298043,0.038738,0.056160,0.357012,104.0,107.0,15.878029,3.382529,-10.863445,3.070357,-1.625993,11.003457,0.0,122.0,198.491637,45.899716,2.037557,0.377780,-1.061196,2.127991,92.0,89.0,16.874552,1.828290,0.091051,0.233792,-0.662657,0.491343,112.0,111.0,23.101795,4.482961,6.491506,8.822068,-1.546606,7.136992,117.0,84.0,23.504732,-2.549069,1588.840909,3011.456099,3.281099,1589.785626,123.0,124.0,831.006682,795.214278,1480.232955,6336.110820,-1.564763,1482.365579,119.0,116.0,1077.919209,604.473679,1564.340909,1984.835017,2.488942,1564.973379,112.0,107.0,903.008146,339.886684,1508.803977,6467.474286,-1.614841,1510.939622,127.0,126.0,694.024024,544.577618,1501.272727,6165.640508,-1.564673,1503.318970,116.0,114.0,1132.811332,1051.223805,1570.758523,2722.246366,2.647024,1571.622362,103.0,102.0,557.741910,388.517307,1
2,2.0,0.010365,0.007636,-0.476359,0.087871,72.0,89.0,2.114065,1.765686,0.022350,0.005579,-0.389757,0.077866,107.0,79.0,1.807463,1.584541,0.145941,0.006013,-0.882224,0.165211,82.0,68.0,0.843492,0.653464,0.020623,0.036905,0.760012,0.192939,118.0,117.0,6.538233,6.238427,0.298244,0.038642,0.064383,0.357046,97.0,100.0,14.328510,3.768672,-10.859006,3.068588,-1.623530,10.998996,0.0,118.0,196.501320,45.222807,2.036657,0.376992,-1.060681,2.126945,92.0,89.0,16.767057,1.737122,0.091558,0.233218,-0.655451,0.490857,105.0,104.0,22.497374,4.301099,6.504136,8.853317,-1.546434,7.150666,1

In [23]:
path_file = path_file.replace(r"M3\FAULT_M3_5", "")
os.chdir(path_file)
df_merged.to_csv('m3_fault5.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
